In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

print("📥 Étape 1 : Chargement du dataset...")
df = pd.read_csv(r"C:\Users\Utilisateur\Desktop\IntroductionRecherche\Dataset\metahateNetoye.csv")
df = df[['label', 'text']]

print("⚖️ Étape 2 : Équilibrage des classes par oversampling...")
max_class_size = df['label'].value_counts().max()
df = df.groupby('label').apply(lambda x: x.sample(max_class_size, replace=True, random_state=42)).reset_index(drop=True)
print("✔️ Classes équilibrées :")
print(df['label'].value_counts())

print("🧹 Étape 3 : Suppression des tweets vides...")
df = df[df['text'].str.strip().astype(bool)].reset_index(drop=True)
print(f"Nombre de tweets après nettoyage : {len(df)}")

print("📊 Étape 4 : Visualisation de la répartition des classes...")
plt.figure(figsize=(6, 4))
sns.countplot(x='label', data=df)
plt.title("📊 Répartition des classes dans le dataset (après équilibrage)")
plt.xlabel("Classe")
plt.ylabel("Nombre d'exemples")
plt.tight_layout()
plt.show()

print("🧠 Étape 5 : Vectorisation des textes (CountVectorizer)...")
vectorizer = CountVectorizer(max_features=700000)
X = vectorizer.fit_transform(df['text'])
y = df['label']
print(f"✅ Shape des données vectorisées : {X.shape}")

print("📂 Étape 6 : Découpage en jeu d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
print(f"✔️ Taille du jeu d'entraînement : {X_train.shape[0]}")
print(f"✔️ Taille du jeu de test : {X_test.shape[0]}")

print("🤖 Étape 7 : Entraînement du modèle (Logistic Regression)...")
model = LogisticRegression(
    C=1000.0,
    penalty='l2',
    solver='lbfgs',
    max_iter=2000,
    class_weight='balanced',
    multi_class='multinomial',
    random_state=42
)
model.fit(X_train, y_train)
print("✅ Modèle entraîné.")
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, classification_report, roc_curve, auc
)

# === Calcul des métriques globales ===
accuracy_global = accuracy_score(y_test, y_pred)
precision_global = precision_score(y_test, y_pred, average='macro')
recall_global = recall_score(y_test, y_pred, average='macro')
f1_macro_global = f1_score(y_test, y_pred, average='macro')
f1_micro_global = f1_score(y_test, y_pred, average='micro')

# === Calcul des métriques par classe ===
classes = np.unique(y_test)
metrics_by_class = []

for cls in classes:
    # Filtrer les données pour la classe actuelle
    idx_cls = (y_test == cls)
    acc_cls = accuracy_score(y_test[idx_cls], y_pred[idx_cls])
    precision_cls = precision_score(y_test, y_pred, labels=[cls], average='macro', zero_division=0)
    recall_cls = recall_score(y_test, y_pred, labels=[cls], average='macro', zero_division=0)
    f1_macro_cls = f1_score(y_test, y_pred, labels=[cls], average='macro', zero_division=0)
    f1_micro_cls = f1_score(y_test, y_pred, labels=[cls], average='micro', zero_division=0)
    auc_cls = roc_auc_score((y_test == cls).astype(int), model.predict_proba(X_test)[:, cls])

    metrics_by_class.append({
        'Classe': cls,
        'Accuracy': acc_cls,
        'Precision': precision_cls,
        'Recall': recall_cls,
        'F1 Macro': f1_macro_cls,
        'F1 Micro': f1_micro_cls,
        'AUC': auc_cls
    })

# Ajouter les métriques globales
metrics_by_class.append({
    'Classe': 'Global',
    'Accuracy': accuracy_global,
    'Precision': precision_global,
    'Recall': recall_global,
    'F1 Macro': f1_macro_global,
    'F1 Micro': f1_micro_global,
    'AUC': 'N/A'  # AUC globale n'est pas calculée ici
})

# Affichage des métriques dans un tableau
df_metrics = pd.DataFrame(metrics_by_class)
df_metrics.set_index('Classe', inplace=True)

print("\n📊 Métriques par classe + Globales :")
print(df_metrics.round(4))

# === Matrice de confusion ===
print("🧾 Étape 9 : Affichage de la matrice de confusion...")
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.title("📊 Matrice de Confusion")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.tight_layout()
plt.show()

# === Courbes ROC ===
print("📉 Étape 10 : Génération des courbes ROC pour les deux classes...")
plt.figure(figsize=(8, 6))

for cls in classes:
    fpr, tpr, _ = roc_curve((y_test == cls).astype(int), model.predict_proba(X_test)[:, cls])
    auc_cls = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Classe {cls} (AUC = {auc_cls:.2f})')

# Ajouter la ligne aléatoire
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--', label='Aléatoire')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('📈 Courbes ROC - Classification Binaire')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()